In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
min_payload = df['Payload Mass (kg)'].min()
max_payload = df['Payload Mass (kg)'].max()
print(min_payload,max_payload)

0.0 9600.0


In [4]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',style={'textAlign':'center','font-size':'20px','color':'black'}),
    html.Div(dcc.Dropdown(id='site-dropdown',options=[
                          {'label':'All Sites','value':'All Sites'},
                          {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                         {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                         {'label':'KSC LC-39A','value':'KSC LC-39A'},
                         {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}],value='All Sites',placeholder='Select a Launch Site',
                          style={'font-size':'12px','color':'red'},searchable=True)
        
    ),
    dcc.RangeSlider(id='payload-slider',
                        min=min_payload, max=max_payload, step=1000,
                        marks={i: f'{i} kg' for i in range(int(min_payload), int(max_payload) + 1000, 1000)},
                        value=[min_payload, max_payload]
                        ),
    dcc.Graph(id='success-pie-chart'),
    dcc.Graph(id='payload-success-scatter-chart')
    
])

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Data for all sites
        success_counts = df['class'].value_counts()
        fig = px.pie(values=success_counts, 
                     names=['Failure', 'Success'], 
                     title='Total Success and Failure for All Sites')
    else:
        # Filter data by selected site
        filtered_df = df[df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts()
        if success_counts.empty:
            success_counts = [0, 0]  # Handle empty dataframe case
        else:
            success_counts = [success_counts.get(0, 0), success_counts.get(1, 0)]  # Ensure correct handling of counts
        fig = px.pie(values=success_counts, 
                     names=['Failure', 'Success'], 
                     title=f'Success and Failure for {selected_site}')
    
    return fig

@app.callback(
    Output(component_id='payload-success-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_plot(selected_site, payload_range):
    filtered_df = df[(df['Payload Mass (kg)'] >= payload_range[0]) & 
                     (df['Payload Mass (kg)'] <= payload_range[1])]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
                     color='Booster Version',
                     title=f'Payload vs. Success for {selected_site}')
    
    return fig
    
if __name__ == '__main__':
    app.run_server(port=8053)
